In [1]:
import pandas as pd

### Data Cleaning & Preparation & Engineering

In [39]:
df = pd.read_csv('bundesliga_cleaned_salaries.csv')

In [40]:
# Drop unnecessary columns
df_cleaned = df.drop(columns=['Unnamed: 0', 'Unnamed: 0_x'])

# Convert the 'date' column to datetime format
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], dayfirst=True)

# Convert 'time' to a proper time format 
df_cleaned['time'] = pd.to_datetime(df_cleaned['time'], format='%H:%M').dt.time

# Display the updated data types and a sample of the cleaned data
df_cleaned.dtypes

date                datetime64[ns]
time                        object
comp                        object
round                       object
day                         object
venue                       object
result                      object
gf                           int64
ga                           int64
xg                         float64
xga                        float64
poss                         int64
attendance                 float64
captain                     object
formation                   object
opp formation               object
referee                     object
match report                object
notes                       object
sh                           int64
sot                          int64
dist                       float64
fk                         float64
pk                           int64
pkatt                        int64
season                       int64
team                        object
opponent                    object
home_team_salary    

### Setting statistics for last 4 games

In [47]:
# Sort by team, season, and date to ensure chronological order
df_cleaned = df_cleaned.sort_values(by=['team', 'season', 'date'])

# Group by team and season, then calculate rolling average excluding the current game using .shift(1)
df_cleaned['gf_last_4_games'] = df_cleaned.groupby(['team', 'season'])['gf'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)
df_cleaned['ga_last_4_games'] = df_cleaned.groupby(['team', 'season'])['ga'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)
# Rolling average for expected goals against (xga)
df_cleaned['xga_last_4_games'] = df_cleaned.groupby(['team', 'season'])['xga'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)
# Calculate points per match for each game (win = 3, draw = 1, loss = 0)
df_cleaned['points'] = df_cleaned['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))

# Calculate rolling points per match over the last 4 games
df_cleaned['avg_points_last_4_games'] = df_cleaned.groupby(['team', 'season'])['points'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)


In [48]:
df_cleaned

,date,time,comp,round,day,venue,result,gf,ga,xg,...,season,team,opponent,home_team_salary,away_team_salary,gf_last_4_games,ga_last_4_games,xga_last_4_games,points,avg_points_last_4_games
2384,2020-09-19,15:30:00,Bundesliga,Matchweek 1,Sat,Away,D,1,1,0.8,...,2021,Arminia,Eintracht Frankfurt,9590000,35180000,NaN,NaN,NaN,1,NaN
2385,2020-09-26,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,0,0.2,...,2021,Arminia,Koln,9590000,29970000,1.000000,1.000000,2.300000,3,1.000000
2386,2020-10-03,15:30:00,Bundesliga,Matchweek 3,Sat,Away,L,0,1,1.1,...,2021,Arminia,Werder Bremen,9590000,29900000,1.000000,0.500000,1.850000,0,2.000000
2387,2020-10-17,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,4,1.4,...,2021,Arminia,Bayern Munich,9590000,224300000,0.666667,0.666667,1.366667,0,1.333333
2388,2020-10-25,15:30:00,Bundesliga,Matchweek 5,Sun,Away,L,1,2,0.5,...,2021,Arminia,Wolfsburg,9590000,57870000,0.750000,1.500000,1.800000,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,2024-05-18,15:30:00,Bundesliga,Matchweek 34,Sat,Home,L,1,3,0.8,...,2024,Wolfsburg,Mainz 05,54830000,28540000,1.500000,0.750000,1.050000,0,2.250000
48,2024-08-25,15:30:00,Bundesliga,Matchweek 1,Sun,Home,L,2,3,1.9,...,2025,Wolfsburg,Bayern Munich,52320000,270230000,NaN,NaN,NaN,0,NaN
49,2024-08-31,15:30:00,Bundesliga,Matchweek 2,Sat,Away,W,2,0,1.3,...,2025,Wolfsburg,Holstein Kiel,52320000,11680000,2.000000,3.000000,2.400000,3,0.000000
50,2024-09-14,15:30:00,Bundesliga,Matchweek 3,Sat,Home,L,1,2,1.5,...,2025,Wolfsburg,Eintracht Frankfurt,52320000,38280000,2.000000,1.500000,1.750000,0,1.500000


#### check if my calculation is correct

In [51]:
team_check_with_points = df_cleaned[df_cleaned['team'] == 'Borussia Dortmund'][['date', 'team', 'gf', 'ga', 'xga', 
                                                                        'gf_last_4_games', 'ga_last_4_games', 
                                                                        'xga_last_4_games', 'points', 
                                                                        'avg_points_last_4_games']]
team_check_with_points

,date,team,gf,ga,xga,gf_last_4_games,ga_last_4_games,xga_last_4_games,points,avg_points_last_4_games
1976,2020-09-19,Borussia Dortmund,3,0,0.6,NaN,NaN,NaN,3,NaN
1977,2020-09-26,Borussia Dortmund,0,2,1.2,3.000000,0.000000,0.600000,0,3.000000
1978,2020-10-03,Borussia Dortmund,4,0,0.5,1.500000,1.000000,0.900000,3,1.500000
1979,2020-10-17,Borussia Dortmund,1,0,1.2,2.333333,0.666667,0.766667,3,2.000000
1980,2020-10-24,Borussia Dortmund,3,0,0.2,2.000000,0.500000,0.875000,3,2.250000
...,...,...,...,...,...,...,...,...,...,...
241,2024-05-18,Borussia Dortmund,4,0,0.3,1.750000,2.250000,2.075000,3,1.000000
28,2024-08-24,Borussia Dortmund,2,0,1.4,NaN,NaN,NaN,3,NaN
29,2024-08-31,Borussia Dortmund,0,0,0.3,2.000000,0.000000,1.400000,1,3.000000
30,2024-09-13,Borussia Dortmund,4,2,1.8,1.000000,0.000000,0.850000,3,2.000000


In [101]:
df_cleaned.to_csv('bundesliga_matches_engineering_step_0.csv')

#### i need more rolling averages

In [2]:
df_cleaned = pd.read_csv('bundesliga_matches_engineering_step_2.csv')

In [3]:
# Calculate rolling averages for the specified columns for the team

# Shots (sh)
df_cleaned['sh_last_4_games'] = df_cleaned.groupby(['team', 'season'])['sh'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Shots on Target (sot)
df_cleaned['sot_last_4_games'] = df_cleaned.groupby(['team', 'season'])['sot'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Distance covered (dist)
df_cleaned['dist_last_4_games'] = df_cleaned.groupby(['team', 'season'])['dist'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Free Kicks (fk)
df_cleaned['fk_last_4_games'] = df_cleaned.groupby(['team', 'season'])['fk'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Penalty Kicks (pk)
df_cleaned['pk_last_4_games'] = df_cleaned.groupby(['team', 'season'])['pk'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Penalty Kick Attempts (pkatt)
df_cleaned['pkatt_last_4_games'] = df_cleaned.groupby(['team', 'season'])['pkatt'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Expected Goals (xg)
df_cleaned['xg_last_4_games'] = df_cleaned.groupby(['team', 'season'])['xg'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)

# Possession (poss)
df_cleaned['poss_last_4_games'] = df_cleaned.groupby(['team', 'season'])['poss'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=1).mean()
)



In [4]:
df_cleaned

,Unnamed: 0.1,Unnamed: 0,date,time,comp,round,day,venue,result,gf,...,opponent_midfield,opponent_defense,sh_last_4_games,sot_last_4_games,dist_last_4_games,fk_last_4_games,pk_last_4_games,pkatt_last_4_games,xg_last_4_games,poss_last_4_games
0,0,0,19.09.2020,15:30:00,Bundesliga,Matchweek 1,Sat,Away,D,1,...,77,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,26.09.2020,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,...,74,72,12.000000,4.000000,22.300000,0.000000,0.000000,0.000000,0.800000,38.000000
2,2,2,03.10.2020,15:30:00,Bundesliga,Matchweek 3,Sat,Away,L,0,...,74,75,9.500000,3.000000,24.300000,0.000000,0.000000,0.000000,0.500000,42.500000
3,3,3,17.10.2020,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,...,85,82,9.333333,3.333333,22.233333,0.000000,0.000000,0.000000,0.700000,49.000000
4,4,4,25.10.2020,15:30:00,Bundesliga,Matchweek 5,Sun,Away,L,1,...,78,77,9.750000,3.750000,21.625000,0.250000,0.000000,0.000000,0.875000,45.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,2515,2515,18.05.2024,15:30:00,Bundesliga,Matchweek 34,Sat,Home,L,1,...,75,74,11.750000,4.500000,22.800000,0.750000,0.000000,0.000000,0.925000,47.250000
2516,2516,2516,25.08.2024,15:30:00,Bundesliga,Matchweek 1,Sun,Home,L,2,...,83,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2517,2517,2517,31.08.2024,15:30:00,Bundesliga,Matchweek 2,Sat,Away,W,2,...,70,68,9.000000,1.000000,19.700000,0.000000,1.000000,1.000000,1.900000,30.000000
2518,2518,2518,14.09.2024,15:30:00,Bundesliga,Matchweek 3,Sat,Home,L,1,...,76,77,11.000000,2.500000,19.000000,0.500000,0.500000,0.500000,1.600000,38.000000


In [5]:
# List of columns to add for opponent stats (new rolling averages)
rolling_avg_columns_to_copy = [
    'sh_last_4_games', 'sot_last_4_games', 'dist_last_4_games', 
    'fk_last_4_games', 'pk_last_4_games', 'pkatt_last_4_games', 
    'xg_last_4_games', 'poss_last_4_games'
]

# Loop over the columns to create the opponent versions
for col in rolling_avg_columns_to_copy:
    opponent_col = f'opponent_{col}'
    df_cleaned[opponent_col] = pd.NA  # Initialize the new opponent columns

    # Iterate through each row to find the matching row for the opponent
    for i, row in df_cleaned.iterrows():
        match = df_cleaned[(df_cleaned['date'] == row['date']) & 
                           (df_cleaned['team'] == row['opponent']) & 
                           (df_cleaned['opponent'] == row['team'])]
        if not match.empty:
            match_idx = match.index[0]
            df_cleaned.at[i, opponent_col] = df_cleaned.at[match_idx, col]


In [6]:
df_cleaned.to_csv('tests2.csv')

### ! Some steps are not in order cause during project development i realised i need more stats so i had to go back from further steps etc.

### Since i want FIFA Ratings (they are estimated by proffesionalls so they are relevant) for every team for every season in Bundesliga and only 1 site provide them (sofifa.com) and the site is hard to WebScrape i will manually make df's using chat gpt just by copying them from site it would be more time efficient than trying to web scrape

In [57]:
# Creating a DataFrame for season 25 (2024/2025 season) from the provided data.

season_25_data = {
    'team': [
        'FC Bayern München', 'Bayer 04 Leverkusen', 'Borussia Dortmund', 'RB Leipzig',
        'SC Freiburg', 'Eintracht Frankfurt', 'VfB Stuttgart', 'TSG Hoffenheim', 
        'VfL Wolfsburg', 'Borussia Mönchengladbach', 'FC Union Berlin', 'Werder Bremen', 
        'FC Augsburg', 'FSV Mainz 05', 'Heidenheim', 'VfL Bochum 1848', 'St. Pauli', 'Holstein Kiel'
    ],
    'Overall': [84, 83, 81, 80, 76, 76, 76, 76, 76, 75, 75, 74, 74, 74, 72, 72, 70, 69],
    'Attack': [90, 82, 81, 81, 78, 76, 80, 76, 76, 76, 75, 75, 74, 74, 71, 71, 70, 69],
    'Midfield': [83, 84, 82, 80, 76, 76, 76, 77, 76, 76, 74, 74, 74, 74, 71, 72, 71, 70],
    'Defense': [82, 82, 81, 80, 77, 77, 74, 73, 75, 74, 75, 74, 74, 74, 73, 72, 70, 68],
    'season': [2025] * 18  # Setting the season as 2025 for all teams
}

# Creating the DataFrame
season_25_df = pd.DataFrame(season_25_data)

# Display the DataFrame
season_25_df


,team,Overall,Attack,Midfield,Defense,season
0,FC Bayern München,84,90,83,82,2025
1,Bayer 04 Leverkusen,83,82,84,82,2025
2,Borussia Dortmund,81,81,82,81,2025
3,RB Leipzig,80,81,80,80,2025
4,SC Freiburg,76,78,76,77,2025
5,Eintracht Frankfurt,76,76,76,77,2025
6,VfB Stuttgart,76,80,76,74,2025
7,TSG Hoffenheim,76,76,77,73,2025
8,VfL Wolfsburg,76,76,76,75,2025
9,Borussia Mönchengladbach,75,76,76,74,2025


In [58]:
# Creating a DataFrame for season 24 (2023/2024 season) from the provided data.

season_24_data = {
    'team': [
        'FC Bayern München', 'Bayer 04 Leverkusen', 'Borussia Dortmund', 'RB Leipzig',
        'Eintracht Frankfurt', 'VfB Stuttgart', 'Borussia Mönchengladbach', 'SC Freiburg',
        'FC Union Berlin', 'TSG Hoffenheim', 'VfL Wolfsburg', 'FSV Mainz 05', 'Werder Bremen', 
        'FC Augsburg', 'FC Köln', 'VfL Bochum 1848', 'Heidenheim', 'Darmstadt 98'
    ],
    'Overall': [84, 82, 81, 80, 77, 77, 76, 76, 76, 76, 76, 75, 74, 74, 73, 73, 72, 71],
    'Attack': [90, 82, 81, 81, 76, 80, 75, 77, 75, 76, 76, 74, 72, 76, 73, 73, 74, 70],
    'Midfield': [84, 83, 82, 80, 77, 75, 77, 76, 75, 77, 77, 75, 74, 74, 72, 74, 72, 71],
    'Defense': [83, 81, 81, 80, 77, 76, 75, 76, 76, 75, 74, 74, 73, 75, 72, 72, 72, 70],
    'season': [2024] * 18  # Setting the season as 2024 for all teams
}


season_24_df = pd.DataFrame(season_24_data)


season_24_df


,team,Overall,Attack,Midfield,Defense,season
0,FC Bayern München,84,90,84,83,2024
1,Bayer 04 Leverkusen,82,82,83,81,2024
2,Borussia Dortmund,81,81,82,81,2024
3,RB Leipzig,80,81,80,80,2024
4,Eintracht Frankfurt,77,76,77,77,2024
5,VfB Stuttgart,77,80,75,76,2024
6,Borussia Mönchengladbach,76,75,77,75,2024
7,SC Freiburg,76,77,76,76,2024
8,FC Union Berlin,76,75,75,76,2024
9,TSG Hoffenheim,76,76,77,75,2024


In [59]:
# Creating a DataFrame for season 23 (2022/2023 season) from the provided data.

season_23_data = {
    'team': [
        'FC Bayern München', 'Borussia Dortmund', 'RB Leipzig', 'Bayer 04 Leverkusen',
        'Eintracht Frankfurt', 'Borussia Mönchengladbach', 'SC Freiburg', 'TSG Hoffenheim',
        'VfL Wolfsburg', 'FC Union Berlin', 'FSV Mainz 05', 'FC Köln', 'VfB Stuttgart', 
        'FC Augsburg', 'Hertha BSC', 'Werder Bremen', 'VfL Bochum 1848', 'Schalke 04'
    ],
    'Overall': [85, 81, 81, 79, 78, 77, 76, 76, 76, 75, 75, 74, 74, 74, 74, 73, 73, 72],
    'Attack': [86, 82, 83, 81, 79, 80, 76, 76, 76, 75, 76, 71, 72, 73, 74, 76, 73, 73],
    'Midfield': [85, 81, 81, 78, 77, 78, 76, 78, 77, 74, 75, 75, 75, 73, 75, 71, 72, 72],
    'Defense': [83, 82, 79, 80, 76, 76, 77, 76, 76, 75, 74, 73, 75, 74, 73, 74, 73, 72],
    'season': [2023] * 18  # Setting the season as 2023 for all teams
}


season_23_df = pd.DataFrame(season_23_data)


season_23_df

,team,Overall,Attack,Midfield,Defense,season
0,FC Bayern München,85,86,85,83,2023
1,Borussia Dortmund,81,82,81,82,2023
2,RB Leipzig,81,83,81,79,2023
3,Bayer 04 Leverkusen,79,81,78,80,2023
4,Eintracht Frankfurt,78,79,77,76,2023
5,Borussia Mönchengladbach,77,80,78,76,2023
6,SC Freiburg,76,76,76,77,2023
7,TSG Hoffenheim,76,76,78,76,2023
8,VfL Wolfsburg,76,76,77,76,2023
9,FC Union Berlin,75,75,74,75,2023


In [61]:
# Creating a DataFrame for season 22 (2021/2022 season) from the provided data.

season_22_data = {
    'team': [
        'FC Bayern München', 'Borussia Dortmund', 'RB Leipzig', 'Bayer 04 Leverkusen',
        'Borussia Mönchengladbach', 'TSG Hoffenheim', 'VfL Wolfsburg', 'Eintracht Frankfurt',
        'SC Freiburg', 'FC Köln', 'VfB Stuttgart', 'FC Augsburg', 'Hertha BSC', 'FSV Mainz 05',
        'FC Union Berlin', 'DSC Arminia Bielefeld', 'VfL Bochum 1848', 'SpVgg Greuther Fürth'
    ],
    'Overall': [84, 81, 80, 79, 78, 77, 77, 76, 75, 75, 74, 74, 74, 74, 73, 72, 72, 71],
    'Attack': [92, 83, 82, 83, 78, 78, 77, 76, 73, 76, 74, 73, 74, 74, 74, 71, 72, 70],
    'Midfield': [85, 81, 81, 79, 77, 76, 78, 77, 74, 75, 74, 74, 75, 74, 73, 72, 71, 70],
    'Defense': [82, 81, 79, 78, 79, 76, 76, 75, 76, 74, 74, 74, 74, 75, 74, 73, 73, 70],
    'season': [2022] * 18  # Setting the season as 2022 for all teams
}


season_22_df = pd.DataFrame(season_22_data)


season_22_df


,team,Overall,Attack,Midfield,Defense,season
0,FC Bayern München,84,92,85,82,2022
1,Borussia Dortmund,81,83,81,81,2022
2,RB Leipzig,80,82,81,79,2022
3,Bayer 04 Leverkusen,79,83,79,78,2022
4,Borussia Mönchengladbach,78,78,77,79,2022
5,TSG Hoffenheim,77,78,76,76,2022
6,VfL Wolfsburg,77,77,78,76,2022
7,Eintracht Frankfurt,76,76,77,75,2022
8,SC Freiburg,75,73,74,76,2022
9,FC Köln,75,76,75,74,2022


In [62]:
# Creating a DataFrame for season 21 (2020/2021 season) from the provided data.

season_21_data = {
    'team': [
        'FC Bayern München', 'Borussia Dortmund', 'RB Leipzig', 'Borussia Mönchengladbach',
        'Bayer 04 Leverkusen', 'VfL Wolfsburg', 'Eintracht Frankfurt', 'TSG Hoffenheim',
        'Hertha BSC', 'SC Freiburg', 'FC Köln', 'Schalke 04', 'Werder Bremen', 'FC Augsburg',
        'VfB Stuttgart', 'FC Union Berlin', 'FSV Mainz 05', 'DSC Arminia Bielefeld'
    ],
    'Overall': [84, 81, 80, 79, 79, 78, 77, 76, 76, 74, 74, 74, 74, 74, 73, 73, 73, 72],
    'Attack': [92, 87, 78, 78, 78, 78, 79, 75, 77, 75, 74, 74, 73, 73, 74, 75, 72, 72],
    'Midfield': [85, 81, 81, 79, 78, 78, 77, 77, 76, 75, 74, 75, 74, 74, 73, 72, 73, 72],
    'Defense': [82, 81, 80, 80, 78, 77, 75, 76, 75, 73, 72, 72, 75, 74, 72, 73, 73, 71],
    'season': [2021] * 18  # Setting the season as 2021 for all teams
}


season_21_df = pd.DataFrame(season_21_data)


season_21_df

,team,Overall,Attack,Midfield,Defense,season
0,FC Bayern München,84,92,85,82,2021
1,Borussia Dortmund,81,87,81,81,2021
2,RB Leipzig,80,78,81,80,2021
3,Borussia Mönchengladbach,79,78,79,80,2021
4,Bayer 04 Leverkusen,79,78,78,78,2021
5,VfL Wolfsburg,78,78,78,77,2021
6,Eintracht Frankfurt,77,79,77,75,2021
7,TSG Hoffenheim,76,75,77,76,2021
8,Hertha BSC,76,77,76,75,2021
9,SC Freiburg,74,75,75,73,2021


In [65]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
all_seasons_fifa_ratings = pd.concat([season_21_df, season_22_df, season_23_df, season_24_df, season_25_df])


all_seasons_fifa_ratings

In [67]:
all_seasons_fifa_ratings.to_csv('all_seasons_fifa_ratings.csv')

### I want to change name in my DF so both DF match

In [92]:
# mapping from df_cleaned team names to Sofifa names (all_seasons_df)
team_name_mapping = {
    'Bayern Munich': 'FC Bayern München',
    'Bayer Leverkusen': 'Bayer 04 Leverkusen',
    'Borussia Dortmund': 'Borussia Dortmund',
    'RB Leipzig': 'RB Leipzig',
    'Freiburg': 'SC Freiburg',
    'Frankfurt': 'Eintracht Frankfurt',
    'Stuttgart': 'VfB Stuttgart',
    'Hoffenheim': 'TSG Hoffenheim',
    'Wolfsburg': 'VfL Wolfsburg',
    'Monchengladbach': 'Borussia Mönchengladbach',
    'Union Berlin': 'FC Union Berlin',
    'Werder Bremen': 'Werder Bremen',
    'Augsburg': 'FC Augsburg',
    'Mainz 05': 'FSV Mainz 05',
    'Koln': 'FC Köln',
    'Bochum': 'VfL Bochum 1848',
    'Hertha': 'Hertha BSC',
    'Arminia': 'DSC Arminia Bielefeld',
    'Greuther Furth': 'SpVgg Greuther Fürth',
    'Schalke': 'Schalke 04',
    'Heidenheim': 'Heidenheim',
    'Holstein Kiel': 'Holstein Kiel',
    'St Pauli': 'St. Pauli',
    'Darmstadt': 'Darmstadt 98'
}

# Apply mapping
df_cleaned['home_team'] = df_cleaned['home_team'].replace(team_name_mapping)



In [93]:
# And the same for away team

df_cleaned['away_team'] = df_cleaned['away_team'].replace(team_name_mapping)

### now since everything works fine i will merge them

In [82]:
df_cleaned_copy = df_cleaned.copy()

In [94]:
df_cleaned.rename(columns={'team': 'home_team', 'opponent': 'away_team'}, inplace=True) ## Renaming so it will be more understandable

In [95]:
# Assuming df_cleaned is already available, we proceed with merging fifa_ratings_df with df_cleaned

# Step 1: Merge Home Team Ratings
df_cleaned = df_cleaned.merge(all_seasons_fifa_ratings, left_on=['home_team', 'season'], right_on=['team', 'season'], how='left')

# Rename the columns for clarity
df_cleaned.rename(columns={
    'Overall': 'home_team_overall', 
    'Attack': 'home_team_attack', 
    'Midfield': 'home_team_midfield', 
    'Defense': 'home_team_defense'
}, inplace=True)

# Drop the redundant 'team' column from the home team merge
df_cleaned.drop(columns=['team'], inplace=True)

# Step 2: Merge Away Team Ratings
df_cleaned = df_cleaned.merge(all_seasons_fifa_ratings, left_on=['away_team', 'season'], right_on=['team', 'season'], how='left')

# Rename the columns for the Away Team
df_cleaned.rename(columns={
    'Overall': 'away_team_overall', 
    'Attack': 'away_team_attack', 
    'Midfield': 'away_team_midfield', 
    'Defense': 'away_team_defense'
}, inplace=True)

# Drop the redundant 'team' column from the away team merge
df_cleaned.drop(columns=['team'], inplace=True)

# Display the first few rows of the updated df_cleaned
df_cleaned.head()

,date,time,comp,round,day,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,home_team,away_team,home_team_salary,away_team_salary,gf_last_4_games,ga_last_4_games,xga_last_4_games,points,avg_points_last_4_games,home_team_overall,home_team_attack,home_team_midfield,home_team_defense,away_team_overall,away_team_attack,away_team_midfield,away_team_defense,home_team_overall,home_team_attack,home_team_midfield,home_team_defense,away_team_overall,away_team_attack,away_team_midfield,away_team_defense,home_team_overall,home_team_attack,home_team_midfield,home_team_defense,away_team_overall,away_team_attack,away_team_midfield,away_team_defense
0,2020-09-19,15:30:00,Bundesliga,Matchweek 1,Sat,Away,D,1,1,0.8,2.3,38,6500.0,Fabian Klos,4-1-4-1,3-4-1-2,Benjamin Brand,Match Report,NaN,12,4,22.3,0.0,0,0,2021,DSC Arminia Bielefeld,Eintracht Frankfurt,9590000,35180000,NaN,NaN,NaN,1,NaN,72.0,72.0,72.0,71.0,77.0,79.0,77.0,75.0,72.0,72.0,72.0,71.0,77.0,79.0,77.0,75.0,72,72,72,71,77,79,77,75
1,2020-09-26,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,0,0.2,1.4,47,5460.0,Fabian Klos,04.03.2003,4-2-3-1,Patrick Ittrich,Match Report,NaN,7,2,26.3,0.0,0,0,2021,DSC Arminia Bielefeld,FC Köln,9590000,29970000,1.000000,1.000000,2.300000,3,1.000000,72.0,72.0,72.0,71.0,NaN,NaN,NaN,NaN,72.0,72.0,72.0,71.0,74.0,74.0,74.0,72.0,72,72,72,71,74,74,74,72
2,2020-10-03,15:30:00,Bundesliga,Matchweek 3,Sat,Away,L,0,1,1.1,0.4,62,300.0,Fabian Klos,4-2-3-1,05.03.2002,Robert Schröder,Match Report,NaN,9,4,18.1,0.0,0,0,2021,DSC Arminia Bielefeld,Werder Bremen,9590000,29900000,1.000000,0.500000,1.850000,0,2.000000,72.0,72.0,72.0,71.0,74.0,73.0,74.0,75.0,72.0,72.0,72.0,71.0,74.0,73.0,74.0,75.0,72,72,72,71,74,73,74,75
3,2020-10-17,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,4,1.4,3.1,34,NaN,Fabian Klos,4-2-3-1,4-2-3-1,Daniel Siebert,Match Report,NaN,11,5,19.8,1.0,0,0,2021,DSC Arminia Bielefeld,FC Bayern München,9590000,224300000,0.666667,0.666667,1.366667,0,1.333333,72.0,72.0,72.0,71.0,NaN,NaN,NaN,NaN,72.0,72.0,72.0,71.0,84.0,92.0,85.0,82.0,72,72,72,71,84,92,85,82
4,2020-10-25,15:30:00,Bundesliga,Matchweek 5,Sun,Away,L,1,2,0.5,2.2,52,4519.0,Fabian Klos,4-2-3-1,4-1-4-1,Manuel Gräfe,Match Report,NaN,9,1,23.8,1.0,0,0,2021,DSC Arminia Bielefeld,VfL Wolfsburg,9590000,57870000,0.750000,1.500000,1.800000,0,1.000000,72.0,72.0,72.0,71.0,NaN,NaN,NaN,NaN,72.0,72.0,72.0,71.0,78.0,78.0,78.0,77.0,72,72,72,71,78,78,78,77


In [100]:
df_cleaned.head(5)

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,xg,xga,poss,attendance,captain,home_team_formation,away_team_formation,referee,sh,sot,dist,fk,pk,pkatt,season,home_team,away_team,home_team_salary,away_team_salary,gf_last_4_games,ga_last_4_games,xga_last_4_games,points,avg_points_last_4_games,home_team_overall,home_team_attack,home_team_midfield,home_team_defense,away_team_overall,away_team_attack,away_team_midfield,away_team_defense
0,0,19.09.2020,15:30:00,Bundesliga,Matchweek 1,Sat,Away,D,1,1,0.8,2.3,38,6500.0,Fabian Klos,4-1-4-1,3-4-1-2,Benjamin Brand,12,4,22.3,0.0,0,0,2021,DSC Arminia Bielefeld,Eintracht Frankfurt,9590000,35180000,NaN,NaN,NaN,1,NaN,72,72,72,71,77,79,77,75
1,1,26.09.2020,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,0,0.2,1.4,47,5460.0,Fabian Klos,04.03.2003,4-2-3-1,Patrick Ittrich,7,2,26.3,0.0,0,0,2021,DSC Arminia Bielefeld,FC Köln,9590000,29970000,1.000000,1.000000,2.300000,3,1.000000,72,72,72,71,74,74,74,72
2,2,03.10.2020,15:30:00,Bundesliga,Matchweek 3,Sat,Away,L,0,1,1.1,0.4,62,300.0,Fabian Klos,4-2-3-1,05.03.2002,Robert Schröder,9,4,18.1,0.0,0,0,2021,DSC Arminia Bielefeld,Werder Bremen,9590000,29900000,1.000000,0.500000,1.850000,0,2.000000,72,72,72,71,74,73,74,75
3,3,17.10.2020,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,4,1.4,3.1,34,NaN,Fabian Klos,4-2-3-1,4-2-3-1,Daniel Siebert,11,5,19.8,1.0,0,0,2021,DSC Arminia Bielefeld,FC Bayern München,9590000,224300000,0.666667,0.666667,1.366667,0,1.333333,72,72,72,71,84,92,85,82
4,4,25.10.2020,15:30:00,Bundesliga,Matchweek 5,Sun,Away,L,1,2,0.5,2.2,52,4519.0,Fabian Klos,4-2-3-1,4-1-4-1,Manuel Gräfe,9,1,23.8,1.0,0,0,2021,DSC Arminia Bielefeld,VfL Wolfsburg,9590000,57870000,0.750000,1.500000,1.800000,0,1.000000,72,72,72,71,78,78,78,77


In [2]:
df_cleaned = pd.read_csv('bundesliga_matches_engineering_step_0.csv')

### Each game appears twice in the dataset, Once where a team is the "home team" and again where the team is the "away team", and i have just "team" stats but not opponent stats. I just have to copy those stats from the corresponding row of the same game

In [3]:
# Create a function to copy opponent's statistics
def add_opponent_stats(df):
    # Create new columns for opponent statistics
    opponent_columns = ['opponent_gf', 'opponent_ga', 'opponent_xg', 'opponent_xga', 
                        'opponent_poss', 'opponent_sh', 'opponent_sot', 'opponent_dist', 
                        'opponent_fk', 'opponent_pk', 'opponent_pkatt']
    
    # Initialize these new columns with NaN values
    for col in opponent_columns:
        df[col] = pd.NA

    # Go through each row and find the matching game to copy stats
    for i, row in df.iterrows():
        # Identify the matching game where the opponent is listed as 'team'
        match = df[(df['date'] == row['date']) & 
                   (df['team'] == row['opponent']) & 
                   (df['opponent'] == row['team'])]

        if not match.empty:
            # Take the first match found (in case of duplicates)
            match_idx = match.index[0]
            
            # Copy the relevant columns from the opponent row to the current row
            df.at[i, 'opponent_gf'] = df.at[match_idx, 'gf']
            df.at[i, 'opponent_ga'] = df.at[match_idx, 'ga']
            df.at[i, 'opponent_xg'] = df.at[match_idx, 'xg']
            df.at[i, 'opponent_xga'] = df.at[match_idx, 'xga']
            df.at[i, 'opponent_poss'] = df.at[match_idx, 'poss']
            df.at[i, 'opponent_sh'] = df.at[match_idx, 'sh']
            df.at[i, 'opponent_sot'] = df.at[match_idx, 'sot']
            df.at[i, 'opponent_dist'] = df.at[match_idx, 'dist']
            df.at[i, 'opponent_fk'] = df.at[match_idx, 'fk']
            df.at[i, 'opponent_pk'] = df.at[match_idx, 'pk']
            df.at[i, 'opponent_pkatt'] = df.at[match_idx, 'pkatt']
    
    return df

# Apply the function to the updated dataframe
df_with_opponent_stats = add_opponent_stats(df_cleaned)




#### did some editing in excel

In [ ]:
df_1 = pd.read_csv('bundesliga_matches_engineering_step_1.csv')

### now i have to do the same for rolling averages

In [19]:
# List of columns to add for opponent stats
additional_columns_to_copy = ['gf_last_4_games', 'ga_last_4_games', 'xga_last_4_games', 'points', 'avg_points_last_4_games']

# That will be faster way to give them names
for col in additional_columns_to_copy:
    opponent_col = f'opponent_{col}'
    df_1[opponent_col] = pd.NA  # Initialize the new opponent columns
    
    # Loop through each row and find the matching row for the opponent team
    for i, row in df_1.iterrows():
        match = df_1[(df_1['date'] == row['date']) & 
                             (df_1['team'] == row['opponent']) & 
                             (df_1['opponent'] == row['team'])]
        if not match.empty:
            match_idx = match.index[0]
            df_1.at[i, opponent_col] = df_1.at[match_idx, col]



In [22]:
# forgot to add captain
df_1['opponent_captain'] = pd.NA  # Initialize the opponent captain column

# Loop through each row and copy the captain information from the opponent team
for i, row in df_1.iterrows():
    match = df_1[(df_1['date'] == row['date']) & 
                         (df_1['team'] == row['opponent']) & 
                         (df_1['opponent'] == row['team'])]
    if not match.empty:
        match_idx = match.index[0]
        df_1.at[i, 'opponent_captain'] = df_1.at[match_idx, 'captain']




#### did some editing in excel (setting columns on positions and little inspection if all is set correctly)

In [7]:
df_1 = pd.read_csv('tests2.csv')

In [26]:
df_1.head(10)

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,opponent_points,opponent_avg_points_last_4_games,team_overall,team_attack,team_midfield,team_defense,opponent_overall,opponent_attack,opponent_midfield,opponent_defense
0,0,19.09.2020,15:30:00,Bundesliga,Matchweek 1,Sat,Away,D,1,1,...,1,NaN,72,72,72,71,77,79,77,75
1,1,26.09.2020,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,0,...,0,0.00,72,72,72,71,74,74,74,72
2,2,03.10.2020,15:30:00,Bundesliga,Matchweek 3,Sat,Away,L,0,1,...,3,1.50,72,72,72,71,74,73,74,75
3,3,17.10.2020,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,4,...,3,2.00,72,72,72,71,84,92,85,82
4,4,25.10.2020,15:30:00,Bundesliga,Matchweek 5,Sun,Away,L,1,2,...,3,1.00,72,72,72,71,78,78,78,77
5,5,31.10.2020,15:30:00,Bundesliga,Matchweek 6,Sat,Home,L,0,2,...,3,2.25,72,72,72,71,81,87,81,81
6,6,07.11.2020,15:30:00,Bundesliga,Matchweek 7,Sat,Away,L,0,5,...,3,2.00,72,72,72,71,73,75,72,73
7,7,21.11.2020,15:30:00,Bundesliga,Matchweek 8,Sat,Home,L,1,2,...,3,3.00,72,72,72,71,79,78,78,78
8,8,28.11.2020,15:30:00,Bundesliga,Matchweek 9,Sat,Away,L,1,2,...,3,1.75,72,72,72,71,80,78,81,80
9,9,05.12.2020,15:30:00,Bundesliga,Matchweek 10,Sat,Home,W,2,1,...,0,1.25,72,72,72,71,73,72,73,73


### Since there are some seasons during Covid and there are no values here for attendance i'll set them to 0

In [8]:
df_1['attendance'].fillna(0, inplace=True)

C:\Users\czarn\AppData\Local\Temp\ipykernel_3600\1050251336.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_1['attendance'].fillna(0, inplace=True)


In [9]:
df_1['attendance'].isna().value_counts()

attendance
False    2520
Name: count, dtype: int64

##### Since every match appears twice in the dataset (once as a home game for one team and once as an away game for the other team), you will:
#####   * Keep only the home games for each team.
#####   * The same match will be represented in the dataset when viewed from the home team’s perspective, so you don’t need the duplicate row for the away team's perspective.


In [10]:
# Filter the dataset to keep only the home games
df_1_homegames = df_1[df_1['venue'] == 'Home']

# Verify by displaying the first few rows to ensure it worked correctly
df_1_homegames.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,xg_last_4_games,poss_last_4_games,opponent_sh_last_4_games,opponent_sot_last_4_games,opponent_dist_last_4_games,opponent_fk_last_4_games,opponent_pk_last_4_games,opponent_pkatt_last_4_games,opponent_xg_last_4_games,opponent_poss_last_4_games
1,1,26.09.2020,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,0,...,0.800,38.00,13.00,6.0,12.000000,0.000000,0.000000,0.000000,2.700000,49.000000
3,3,17.10.2020,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,4,...,0.700,49.00,19.00,7.0,16.966667,1.333333,0.666667,0.666667,2.966667,66.666667
5,5,31.10.2020,15:30:00,Bundesliga,Matchweek 6,Sat,Home,L,0,2,...,0.800,48.75,16.75,6.0,16.200000,0.500000,0.000000,0.000000,1.925000,68.500000
7,7,21.11.2020,15:30:00,Bundesliga,Matchweek 8,Sat,Home,L,1,2,...,0.625,40.00,13.50,5.5,19.775000,0.000000,0.250000,0.250000,1.350000,61.500000
9,9,05.12.2020,15:30:00,Bundesliga,Matchweek 10,Sat,Home,W,2,1,...,0.550,36.75,12.25,5.0,17.250000,0.250000,0.500000,0.500000,2.300000,37.000000


In [11]:
df_1_homegames.to_csv('tests.csv')

In [12]:
numeric_features = df_1_homegames.select_dtypes(include=['float64', 'int64'])

# Calculate the variance for each numeric feature
variance = numeric_features.var()

# Display the variance sorted from highest to lowest
variance_sorted = variance.sort_values(ascending=False)
print(variance_sorted)

opponent_team_salary           2.934741e+15
team_salary                    2.844931e+15
attendance                     5.258092e+08
Unnamed: 0                     5.299766e+05
poss                           1.201201e+02
                                   ...     
opponent_pk                    1.020935e-01
opponent_pkatt_last_4_games    4.462826e-02
pkatt_last_4_games             4.397610e-02
opponent_pk_last_4_games       3.700905e-02
pk_last_4_games                3.291800e-02
Length: 61, dtype: float64


In [13]:
df_1_homegames['date'] = pd.to_datetime(df_1_homegames['date'], format='%d.%m.%Y')
df_1_homegames['time'] = pd.to_datetime(df_1_homegames['time'], format='%H:%M:%S').dt.time

# Converting relevant columns to categorical types
categorical_columns = ['comp', 'round', 'day', 'venue', 'result', 'captain', 'opponent_captain', 
                       'home_team_formation', 'away_team_formation', 'referee']

df_1_homegames[categorical_columns] = df_1_homegames[categorical_columns].astype('category')

# Ensuring numeric columns are in correct numeric format
numeric_columns = ['gf', 'ga', 'xg', 'xga', 'poss', 'opponent_gf', 'opponent_ga', 'opponent_xg', 
                   'opponent_xga', 'opponent_poss', 'attendance', 'sh', 'sot', 'dist', 'fk', 'pk', 
                   'team_overall', 'team_attack', 'team_midfield', 'team_defense', 
                   'opponent_overall', 'opponent_attack', 'opponent_midfield', 'opponent_defense']

df_1_homegames[numeric_columns] = df_1_homegames[numeric_columns].apply(pd.to_numeric, errors='coerce')

C:\Users\czarn\AppData\Local\Temp\ipykernel_3600\827512030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_homegames['date'] = pd.to_datetime(df_1_homegames['date'], format='%d.%m.%Y')
C:\Users\czarn\AppData\Local\Temp\ipykernel_3600\827512030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_homegames['time'] = pd.to_datetime(df_1_homegames['time'], format='%H:%M:%S').dt.time
C:\Users\czarn\AppData\Local\Temp\ipykernel_3600\827512030.py:8: SettingWithCopyWarning: 
A value is trying to be se

In [14]:
df_1_homegames.drop(columns=['Unnamed: 0'], inplace=True)

C:\Users\czarn\AppData\Local\Temp\ipykernel_3600\1660839878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_homegames.drop(columns=['Unnamed: 0'], inplace=True)


In [15]:
df_1_homegames.to_csv('bundesliga_matches_engineering_step_3_1.csv')

In [16]:
df_1_homegames = pd.read_csv('bundesliga_matches_engineering_step_3_1.csv')

In [17]:
df_1_homegames

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,xg_last_4_games,poss_last_4_games,opponent_sh_last_4_games,opponent_sot_last_4_games,opponent_dist_last_4_games,opponent_fk_last_4_games,opponent_pk_last_4_games,opponent_pkatt_last_4_games,opponent_xg_last_4_games,opponent_poss_last_4_games
0,1,2020-09-26,15:30:00,Bundesliga,Matchweek 2,Sat,Home,W,1,0,...,0.800,38.00,13.00,6.00,12.000000,0.000000,0.000000,0.000000,2.700000,49.000000
1,3,2020-10-17,18:30:00,Bundesliga,Matchweek 4,Sat,Home,L,1,4,...,0.700,49.00,19.00,7.00,16.966667,1.333333,0.666667,0.666667,2.966667,66.666667
2,5,2020-10-31,15:30:00,Bundesliga,Matchweek 6,Sat,Home,L,0,2,...,0.800,48.75,16.75,6.00,16.200000,0.500000,0.000000,0.000000,1.925000,68.500000
3,7,2020-11-21,15:30:00,Bundesliga,Matchweek 8,Sat,Home,L,1,2,...,0.625,40.00,13.50,5.50,19.775000,0.000000,0.250000,0.250000,1.350000,61.500000
4,9,2020-12-05,15:30:00,Bundesliga,Matchweek 10,Sat,Home,W,2,1,...,0.550,36.75,12.25,5.00,17.250000,0.250000,0.500000,0.500000,2.300000,37.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,2511,2024-04-20,15:30:00,Bundesliga,Matchweek 30,Sat,Home,W,1,0,...,1.300,53.00,16.50,3.75,16.225000,0.750000,0.000000,0.000000,1.075000,53.000000
1256,2513,2024-05-04,15:30:00,Bundesliga,Matchweek 32,Sat,Home,W,3,0,...,0.725,51.50,13.50,3.75,21.025000,0.250000,0.000000,0.000000,1.025000,45.250000
1257,2515,2024-05-18,15:30:00,Bundesliga,Matchweek 34,Sat,Home,L,1,3,...,0.925,47.25,14.00,4.00,16.050000,0.750000,0.000000,0.000000,1.525000,46.250000
1258,2516,2024-08-25,15:30:00,Bundesliga,Matchweek 1,Sun,Home,L,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
